In [1]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import nibabel as nib
import numpy as np
from numba import jit
from scipy.spatial.distance import cdist
from scipy.spatial import cKDTree
from scipy.sparse import csr_matrix
import anndata
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
base_path = '/mnt/Data16Tc/home/haichao/code/SpaCon/'

mus = 'mouse_1'

### 1. Load the data

In [2]:
# load the gene expression h5ad

# adata = sc.read_h5ad(f'{base_path}Data/N_20231213_zxw/{mus}/adata_processed.h5ad')
# adata

adata = sc.read_h5ad(f'/mnt/Data16Tc/home/haichao/code/SpaCon/Data/N_20231213_zxw/{mus}/adata_processed.h5ad')
# add allen region
# allen_region = pd.read_csv(f'/mnt/Data16Tc/home/haichao/code/SpaCon/Data/N_20231213_zxw/{mus}/allen_region.csv')
# adata.obs['region'] = allen_region['region'].to_list()
# add cell type
meta = pd.read_csv(f'/mnt/Data16Tc/home/haichao/code/SpaCon/Data/N_20231213_zxw/{mus}/cell_metadata_with_cluster_annotation.csv')
meta = meta.set_index('cell_label')
meta = meta.loc[adata.obs.index.to_list()]
adata.obs['cell_type'] = meta['class'].to_list()

# adata = adata[adata.obs['cell_type'].str.contains('Glut')]
adata.obs

,brain_section_label,x,y,z,x_section_mean,x_ccf,y_ccf,z_ccf,cell_type
cell_label,,,,,,,,,
99253477382922973735936516512751438783,Zhuang-ABCA-1.089,79.119126,34.160780,5.190517,78.263674,7.911913,3.416078,0.519052,13 CNU-HYa Glut
193467560145029549698621814618068646692,Zhuang-ABCA-1.089,79.276931,35.561997,6.405416,78.263674,7.927693,3.556200,0.640542,01 IT-ET Glut
143795643695719616863922826907770581831,Zhuang-ABCA-1.089,79.329301,39.738416,6.680599,78.263674,7.932930,3.973842,0.668060,01 IT-ET Glut
146348436271229422415293281643148356382,Zhuang-ABCA-1.089,79.333072,37.477056,6.659923,78.263674,7.933307,3.747706,0.665992,01 IT-ET Glut
278645763488428414844834985302551688241,Zhuang-ABCA-1.089,79.314257,37.072463,6.509415,78.263674,7.931426,3.707246,0.650941,01 IT-ET Glut
...,...,...,...,...,...,...,...,...,...
221512805930945316853621272768045586446,Zhuang-ABCA-1.110,96.704595,29.012143,12.448145,97.196243,9.670460,2.901214,1.244814,01 IT-ET Glut
249134686993035920144179729210645112897,Zhuang-ABCA-1.110,96.709790,28.696808,12.887360,97.196243,9.670979,2.869681,1.288736,01 IT-ET Glut
290196967067945916959837635601986015416,Zhuang-ABCA-1.110,96.702707,40.183431,10.158557,97.196243,9.670271,4.018343,1.015856,03 OB-CR Glut


In [5]:
del adata.obs['x_section_mean']
adata.obs

,brain_section_label,x,y,z,x_ccf,y_ccf,z_ccf,cell_type
cell_label,,,,,,,,
99253477382922973735936516512751438783,Zhuang-ABCA-1.089,79.119126,34.160780,5.190517,7.911913,3.416078,0.519052,13 CNU-HYa Glut
193467560145029549698621814618068646692,Zhuang-ABCA-1.089,79.276931,35.561997,6.405416,7.927693,3.556200,0.640542,01 IT-ET Glut
143795643695719616863922826907770581831,Zhuang-ABCA-1.089,79.329301,39.738416,6.680599,7.932930,3.973842,0.668060,01 IT-ET Glut
146348436271229422415293281643148356382,Zhuang-ABCA-1.089,79.333072,37.477056,6.659923,7.933307,3.747706,0.665992,01 IT-ET Glut
278645763488428414844834985302551688241,Zhuang-ABCA-1.089,79.314257,37.072463,6.509415,7.931426,3.707246,0.650941,01 IT-ET Glut
...,...,...,...,...,...,...,...,...
221512805930945316853621272768045586446,Zhuang-ABCA-1.110,96.704595,29.012143,12.448145,9.670460,2.901214,1.244814,01 IT-ET Glut
249134686993035920144179729210645112897,Zhuang-ABCA-1.110,96.709790,28.696808,12.887360,9.670979,2.869681,1.288736,01 IT-ET Glut
290196967067945916959837635601986015416,Zhuang-ABCA-1.110,96.702707,40.183431,10.158557,9.670271,4.018343,1.015856,03 OB-CR Glut


### 2. the NT and ST coordinate match by section

In [6]:
NT_position = np.load(f'{base_path}Data/NT_Data/coordinate_information/position.npy').T   # (421050, 3)
kdtree = cKDTree(NT_position)

for section in tqdm(adata.obs['brain_section_label'].unique()):
    adata_temp = adata[adata.obs['brain_section_label'] == section]
    coordinate = np.array([adata_temp.obs['x'].values, adata_temp.obs['y'].values, adata_temp.obs['z'].values]).T

    # get the index_list matching the transcriptome data
    st2nt_index = []
    for coord in coordinate:
        # 查询最近邻索引
        _, nearest_index = kdtree.query(coord)
        st2nt_index.append(nearest_index)
        
    # save the index_list
    Selected_NT_position = NT_position[st2nt_index]
    index_path = f'./data/{mus}/st2nt_index/'
    os.makedirs(index_path, exist_ok=True)
    np.save(f'{index_path}/{section}_index_list.npy', np.array(st2nt_index))
    np.save(f'{index_path}/{section}_Selected_NT_position.npy', Selected_NT_position)

100%|██████████| 129/129 [00:15<00:00,  8.45it/s]


### 3. Merge cell by section

In [7]:
df = pd.DataFrame()
# for every section and search the NT index
for section in tqdm(adata.obs['brain_section_label'].unique()):
    # print(section)
    # result_dic_file = result_dic_files[0]
    adata_temp = adata[adata.obs['brain_section_label'] == section]

    # load the index file && nt position file
    index_path = f'./data/{mus}/st2nt_index/'
    # os.makedirs(index_path, exist_ok=True)
    st2nt_index = np.load(f'{index_path}/{section}_index_list.npy')
    st2nt_position = np.load(f'{index_path}/{section}_Selected_NT_position.npy')

    # build a original dataframe
    df_tep = pd.DataFrame(data=adata_temp.X.toarray(), index=adata_temp.obs_names, columns=adata_temp.var_names)
    # df_tep['x_section_mean'] = adata_temp.obs['x_section_mean']
    df_tep['x'] = adata_temp.obs['x']
    df_tep['y'] = adata_temp.obs['y']
    df_tep['z'] = adata_temp.obs['z']
    df_tep['NT_index'] = st2nt_index
    df_tep['NT_index'] = df_tep['NT_index'].astype('category')

    # get the ordered cellid
    cell_id = df_tep.groupby("NT_index").apply(lambda x: '_'.join(map(str, x.index)))
    df_cellid = pd.DataFrame(cell_id, columns=['cell_id'])
    # sum and mean the same index data
    df_gene = df_tep.groupby(['NT_index'])[df_tep.columns.difference(['x', 'y', 'z', 'NT_index'])].sum()
    df_xyz = df_tep.groupby(['NT_index']).agg({'x': 'mean', 'y': 'mean', 'z': 'mean'})

    df_tep = pd.concat([df_gene, df_xyz, df_cellid], axis=1)
    df_tep['section'] = section
    # concatenate two dataframe
    df = pd.concat([df, df_tep], axis=0)

100%|██████████| 129/129 [00:15<00:00,  8.54it/s]


In [9]:
# trans nan to 0
df.fillna(0, inplace=True)
# dataframe to anndata
# x_section_mean = df['x_section_mean'].values.copy()
X = df['x'].values.copy()
Y = df['y'].values.copy()
Z = df['z'].values.copy()
sec = df['section'].values.copy()
cell_id = df['cell_id'].values.copy()
del df['x'], df['y'], df['z'], df['cell_id'], df['section']
sparse_matrix = csr_matrix(df.values)

adata_merge = anndata.AnnData(X=sparse_matrix)
adata_merge.obs_names = df.index.to_frame().values.squeeze().astype(str)
adata_merge.var_names = df.columns.to_frame().values.squeeze().astype(str)
# adata_merge.obs['x_section_mean'] = x_section_mean
adata_merge.obs['x'] = X
adata_merge.obs['y'] = Y
adata_merge.obs['z'] = Z
adata_merge.obs['section'] = sec
adata_merge.obs['NT_index'] = adata_merge.obs_names
adata_merge.obs['Cells_id'] = cell_id

In [10]:
adata_merge

AnnData object with n_obs × n_vars = 142030 × 1122
    obs: 'x', 'y', 'z', 'section', 'NT_index', 'Cells_id'

In [ ]:
def plot_section( xx, yy, cc=None, val=None, fig_width = 8, fig_height = 8, cmap=None ) :
    fig, ax = plt.subplots()
    fig.set_size_inches(fig_width, fig_height)
    if cmap is not None :
        plt.scatter(xx,yy,s=30,c=val,marker='.',cmap=cmap)
    elif cc is not None :
        plt.scatter(xx,yy,s=30,color=cc,marker='.')
    ax.set_ylim(11,0)
    ax.set_xlim(0,11)
    ax.axis('equal')
    ax.set_xticks([])
    ax.set_yticks([])
    return fig, ax
for s in adata_merge.obs['section'].unique():
    adata_temp = adata_merge[adata_merge.obs['section'] == s]
    fig, ax = plot_section(adata_temp.obs['z'], adata_temp.obs['y'], cc='b')
    fig.savefig(f'./data/{mus}/mousebrain_fig_merge_section/{s}.png')

In [12]:
adata_merge.write_h5ad(f'./data/{mus}/adata_merge.h5ad', compression='gzip')

### 4. extract the NT adj mat

In [2]:
reference_excel = pd.read_csv(f'{base_path}Data/NT_Data/coordinate_information/region_information.csv',index_col=0)
reference_excel = reference_excel.fillna(0)  # nan->0

# load the number_sum file (1080,)
number_sum_list = np.load(f'{base_path}Data/NT_Data/coordinate_information/number_sum_list.npy')

# load the index_list file
adata_merge = anndata.read(f'./data/{mus}/adata_merge.h5ad')

st2nt_index = np.array(adata_merge.obs['NT_index']).astype(int)   # (203860,)
# save the original index_list (to clip the data)
   # 107909
del adata_merge
print(st2nt_index.shape)
st2nt_index

(250535,)


array([ 17372,  17373,  17374, ..., 394009, 394016, 394023])

In [3]:
# correspoding_NT_data = [None]*st2nt_index.shape[0]
# append_successful_flag = [None]*st2nt_index.shape[0]
# st2nt_index_batch = st2nt_index[:130000]  #
st2nt_index_original = st2nt_index.copy()
shape = (len(st2nt_index), len(st2nt_index))
correspoding_NT_data = np.zeros(shape)
append_successful_flag = np.zeros(st2nt_index.shape[0], dtype=bool)
print('Left brain neuro tracing data extract!')
# left
for j in tqdm(range(reference_excel.shape[1])):  # 12
    for i in range(reference_excel.shape[0]):  # 45
        # update the index_list
        # minus the number_sum_list data before
        if (i+j*45-1) >=0:
            st2nt_index = st2nt_index - number_sum_list[i+j*45-1]
        # if nan, continue
        if reference_excel.values[i,j]==0: # reference_excel.values[i,j] == 'nan':
            continue
        elif number_sum_list[i+j*45]==0: # reference_excel.values[i,j] == 'nan':
            continue
        else:
            # path + column's name + value's name
            temp_neurotracing_data = np.load(f'{base_path}Data/NT_Data/connection_matrix/'+reference_excel.columns.values[j]+'/L_'+reference_excel.values[i,j]+'_projection.npy')
            # clip the data (x,42w)->(x,34103)
            temp_neurotracing_data = temp_neurotracing_data[:,st2nt_index_original]
            # search every index, if it in or not
            for k in range(st2nt_index.shape[0]):  # 34103
                # if the index are within the scope of temp_neurotracing_data indexes
                if (st2nt_index[k]>=0) and (st2nt_index[k]<=(temp_neurotracing_data.shape[0]-1)):
                    # cat the data to the correspoding_NT_data
                    # correspoding_NT_data.append(temp_neurotracing_data[st2nt_index[k]])
                    correspoding_NT_data[k] = temp_neurotracing_data[st2nt_index[k]]
                    # add the flag
                    append_successful_flag[k] = True
                else:
                    continue

# mid_shape = np.array(correspoding_NT_data).shape[0]


print('Right brain neuro tracing data extract!')
# right
for j in tqdm(range(reference_excel.shape[1])): # 12
    for i in range(reference_excel.shape[0]):  # 45
        # update the index_list
        st2nt_index = st2nt_index - number_sum_list[i+j*45+540-1]
        # if nan, continue
        if reference_excel.values[i,j]==0: # reference_excel.values[i,j] == 'nan':
            continue
        elif number_sum_list[i+j*45+540]==0:  # shape==0
            continue
        else:
            # path + column's name + value's name
            temp_neurotracing_data = np.load(f'{base_path}Data/NT_Data/connection_matrix/'+reference_excel.columns.values[j]+'/R_'+reference_excel.values[i,j]+'_projection.npy')
            # clip the data (x,42w)->(x,34103)
            temp_neurotracing_data = temp_neurotracing_data[:, st2nt_index_original]
            # search every index, if it in or not
            for k in range(st2nt_index.shape[0]):  # 34103
                if (st2nt_index[k] >= 0) and (st2nt_index[k] <= (temp_neurotracing_data.shape[0] - 1)):
                    # cat the data to the correspoding_NT_data
                    # correspoding_NT_data.append(temp_neurotracing_data[index_list[k]])
                    correspoding_NT_data[k] = temp_neurotracing_data[st2nt_index[k]]
                    # add the flag
                    append_successful_flag[k] = True
                else:
                    continue

Left brain neuro tracing data extract!


100%|██████████| 12/12 [2:29:32<00:00, 747.67s/it] 


Right brain neuro tracing data extract!


100%|██████████| 12/12 [3:07:21<00:00, 936.80s/it]  


In [4]:
np.save('/mnt/Data18Td/Data/haichao/mouse_connect_data/NT/zxw/mouse_1/zxw_not_symmetric_adj.npy', correspoding_NT_data)

In [4]:
# correspoding_NT_data = np.load('./data/mouse_1/zxw_adj.npy')
# symmetric matrix(max)
for i in tqdm(range(correspoding_NT_data.shape[0])):
    for j in range(i+1,correspoding_NT_data.shape[0]):
        correspoding_NT_data[i,j] = correspoding_NT_data[j,i] = np.maximum(correspoding_NT_data[i,j],correspoding_NT_data[j,i])

np.save('/mnt/Data18Td/Data/haichao/mouse_connect_data/NT/zxw/mouse_1/zxw_adj.npy', correspoding_NT_data)

100%|██████████| 142030/142030 [6:09:48<00:00,  6.40it/s]  
